<a href="https://colab.research.google.com/github/vvivvi/kaggle-c1/blob/master/Kaggle_C1_text_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm 
import gc
import re
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline 


for p in [np, pd, scipy, sklearn, lightgbm]:
    print (p.__name__, p.__version__)

numpy 1.18.1
pandas 0.25.3
scipy 1.4.1
sklearn 0.22.1
lightgbm 2.3.1


In [2]:
def write_predictions_by_array(array, filename):
  df=pd.DataFrame(array)
  df.columns=['item_cnt_month']
  df.to_csv(os.path.join(DATA_FOLDER, filename), index_label='ID')

In [3]:
def clipped_rmse(gt, predicted,clip_min=0, clip_max=20):
  target=np.minimum(np.maximum(gt,clip_min), clip_max)
  return np.sqrt((target-predicted)**2).mean()

In [4]:
#from itertools import product
#import gc
#from tqdm import tqdm_notebook


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [5]:
import os

#!wget -O competitive-data-science-predict-future-sales.zip https://github.com/vvivvi/kaggle-c1/blob/master/competitive-data-science-predict-future-sales.zip?raw=true
#!mkdir competitive-data-science-predict-future-sales
#!unzip -o competitive-data-science-predict-future-sales.zip -d competitive-data-science-predict-future-sales

DATA_FOLDER = 'competitive-data-science-predict-future-sales'

#sales    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
#item_cats = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
#shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
#sample_submission = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))


In [6]:
test_shops=test_spec['shop_id'].unique()
test_items=test_spec['item_id'].unique()
date_block_val = 33
date_block_test = 35 # Dec 2015
#sales = sales[sales['shop_id'].isin(test_shops)]

In [7]:
#all_data.to_csv('all_data_with_category_targets.csv')
all_data=pd.read_csv(DATA_FOLDER + '/all_data_with_category_targets.csv')
all_data=downcast_dtypes(all_data)
gc.collect()

44

In [8]:
cols_to_rename = [col for col in all_data.columns.values if re.search('target',col)] 
cols_to_rename

['target',
 'target_shop',
 'target_item',
 'target_category',
 'target_category_tfidf_unigram_32',
 'target_category_tfidf_unigram_256',
 'target_category_tfidf_bigram_32',
 'target_category_tfidf_bigram_256',
 'target_category_frequent_32',
 'target_category_frequent_256',
 'target_category_within_shop',
 'target_category_tfidf_unigram_32_within_shop',
 'target_category_tfidf_unigram_256_within_shop',
 'target_category_tfidf_bigram_32_within_shop',
 'target_category_tfidf_bigram_256_within_shop',
 'target_category_frequent_32_within_shop',
 'target_category_frequent_256_within_shop',
 'target_item_within_shop']

In [9]:
# List of columns that we will use to create lags
# cols_to_rename = list(all_data.columns.difference(index_cols)) 
cols_to_rename = [col for col in all_data.columns.values if re.search('target',col)] 
index_cols = ['shop_id', 'item_id', 'date_block_num']
shift_range = [2, 3, 4, 5,6,12,13,14] 

for col in cols_to_rename:
    print(col)
    all_data=pd.read_csv(DATA_FOLDER + '/all_data_with_category_targets.csv')
    all_data=all_data[index_cols+[col]]
    all_data=downcast_dtypes(all_data)
    gc.collect()
    for month_shift in tqdm_notebook(shift_range):
        train_shift = all_data[index_cols + [col]].copy()
        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x==col else x
        train_shift = train_shift.rename(columns=foo)

        all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
        all_data=downcast_dtypes(all_data)

        del train_shift
        gc.collect()

        # Don't use old data without enough lags in the past
    all_data = all_data[all_data['date_block_num'] >= 14] 
    all_data.to_csv(DATA_FOLDER + '/' + col + '_lagged.csv')
    
    del all_data
    gc.collect()



target


/home/vvi/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]



target_shop



target_item



target_category


...


target_category_tfidf_bigram_32_within_shop



target_category_tfidf_bigram_256_within_shop



target_category_frequent_32_within_shop



target_category_frequent_256_within_shop



target_item_within_shop


In [ ]:
# List of columns that we will use to create lags
# cols_to_rename = list(all_data.columns.difference(index_cols)) 
cols_to_rename = [col for col in all_data.columns.values if re.search('target',col)] 

index_triplet_to_row_num = {}
for idx,row in all_data.reset_index().iterrows():
    if idx % 10000==1:
        print(idx)
    index_triplet_to_row_num[(row['item_id'], row['shop_id'], row['date_block_num'])] = idx

index_cols = ['shop_id', 'item_id', 'date_block_num']

shift_range = [2, 3, 4, 5,6,12,13,14] 
# there's one month gap between and test periods

for col in cols_to_rename:
    print(col)
    df_result=all_data[index_cols].copy()
    for month_shift in tqdm_notebook(shift_range):
        print(month_shift)
        colname_new = '{}_lag_{}'.format(col, month_shift)
        df_result[colname_new] = 0
        for idx,row in all_data.iterrows():
            row_num_new = index_triplet_to_row_num.get((row['item_id'], row['shop_id'], row['date_block_num']+month_shift),-1)
            if row_num_new >= 0:
                df_result.iloc[row_num_new][colname_new] = row[col]
    
    df_result.to_csv(DATA_FOLDER + '/' + col + '_lagged.csv')
    
    del df_result
    gc.collect()

